In [1]:
%matplotlib inline
import os
import numpy
import rasterio
import pandas
from rasterio.plot import show
data_path = '..'

In [2]:
inun_map = os.path.join(data_path,'data','inundation','inundation_map.tif')
landuse_map = os.path.join(data_path,'data','landuse','landuse_map.tif')

In [3]:
curve_path = os.path.join(data_path,'data','curves','curves.csv')
maxdam_path = os.path.join(data_path,'data','curves','maxdam.csv')

In [78]:
loss_df,damagemap,landuse,inundation = RasterScanner(landuse_map,inun_map,curve_path,maxdam_path,cellsize=25,save=True,
                                                     in_millions=True,output_path=data_path,scenario_name='test',crs={'init': 'EPSG:28992'})

In [80]:
not os.path.exists(data_path)

False